In [1]:
import pandas as pd
import numpy as np
from geopy import geocoders as gc
def centralize(series):
    m, sigma = series.mean(), series.std()
    return (series-m)/sigma

In [2]:
fd = pd.read_excel('FD.xlsx')
fd = fd.iloc[(fd.year.between(2010, 2019)).to_list(),:] \
        .groupby(['country', 'ifs', 'imf_income']).aggregate({'FD': 'mean'}).reset_index()

In [3]:
ed = pd.read_csv('ED.csv').rename(columns = {'Unnamed: 9': 'ED', 'Country Code': 'ifs'}) \
                        .query('`Indicator Code` == "total_theil"')
ed.loc[:, 'ED'] = ed.loc[:,['2011', '2012', '2013', '2014']].apply('mean', axis = 1)
ed = ed.loc[:, ['Country Name', 'ifs', 'ED']]

In [4]:
data = ed.merge(fd, on = 'ifs').drop('Country Name', axis = 1)

In [5]:
data.loc[:, 'ED'] = -1*centralize(data.loc[:, 'ED'])
data.loc[:, 'FD'] = centralize(data.loc[:, 'FD'])
data.loc[:, 'score'] = (data.ED + data.FD)/2

In [6]:
em = data.sort_values(by = 'score', ascending = False).loc[data.loc[:, 'imf_income'] == 'EM'].reset_index(drop = True)

In [7]:
av = data.groupby(['imf_income']).agg({'score':'mean'})
in_eu = ['Bulgaria', 'Croatia', 'Poland', 'Bosnia and Herzegovina', 'Romania', 'Hungary', 'North Macedonia']

In [8]:
em = em.iloc[[(i not in in_eu) and (j > av.loc['EM'].values.item()) for i, j in zip(em.country, em.score)], [2,1,4,5]] \
.reset_index(drop = True)
em

,country,ED,FD,score
0,Thailand,1.288383,1.613700,1.451041
1,China,1.272366,1.179024,1.225695
2,Turkey,1.457791,0.726119,1.091955
3,Malaysia,0.691675,1.486783,1.089229
4,Brazil,0.804054,1.343082,1.073568
5,South Africa,0.765030,0.991371,0.878201
6,India,1.092244,0.558943,0.825593
7,Jordan,1.035307,0.294491,0.664899
8,Panama,0.832553,0.398709,0.615631
9,Mexico,0.880720,0.292634,0.586677


In [9]:
em.to_excel('em.xlsx')

In [9]:
fd

,country,ifs,imf_income,FD
0,AM,11,AM,0.641418
1,Africa,14,Africa,0.148326
2,Albania,914,EM,0.200531
3,Algeria,612,EM,0.132419
4,All countries,1,All countries,0.317279
...,...,...,...,...
187,"Venezuela, RB",299,EM,0.248332
188,Vietnam,582,EM,0.366168
189,Western Hemisphere,18,Western Hemisphere,0.317048
190,"Yemen, Rep.",474,LIC,0.109915
